In [1]:
import Trough_GUI

In [2]:
Trough_GUI.Monitor_Calibrate.Monitor_Setup_Trough(Trough_GUI.calibrations)

In [ ]:
Trough_GUI.status_widgets.update_status({'bal_raw':1.27,'bal_dev':0.02,
                                         'barr_raw':0.8,'barr_dev':0.002,
                                         'temp_raw':3.30,'temp_dev':0.01,
                                         'messages':['No updates',]}, cals)

In [ ]:
cals.barriers.cal_apply([0.7613993559219399, 0.7613712807244503, 0.7614248322753392],
                       [2.684617327574853e-05, 2.057942843404166e-05, 2.3285347508477762e-05])

In [ ]:
from multiprocessing import Process
isinstance(TROUGH,Process)

In [ ]:
TROUGH.is_alive()

In [3]:
import Trough_Control
if not Trough_Control.trough_util.is_trough_initialized():
    cmdsend, datarcv, TROUGH = Trough_Control.trough_util.init_trough()

['Starting Motor Calibration. Please wait...']
['Trough ready']


In [4]:
# Asynchronous update of status widgets
# TODO should this include the calibrations or simply be an asynchronous GUI
import threading
import Trough_GUI

def update_status(cmdsend, datarcv, cals):
    import time
    datapkg = []
    run = True
    update = 0
    while run:
        min_next_time = time.time() + 2.0
        cmdsend.send(['Send',''])
        waiting = True
        while waiting:
            if datarcv.poll():
                datapkg =datarcv.recv()
                update_dict = {'barr_raw':datapkg[1][-1],
                               'barr_dev':datapkg[2][-1],
                               'bal_raw':datapkg[3][-1],
                               'bal_dev':datapkg[4][-1],
                               'temp_raw':datapkg[5][-1],
                               'temp_dev':datapkg[6][-1],
                               'messages':datapkg[7]}
                Trough_GUI.status_widgets.update_status(update_dict, cals)
                waiting = False
        if time.time()< min_next_time:
            time.sleep(min_next_time - time.time())
    return

thread = threading.Thread(target=update_status, args=(cmdsend, datarcv, Trough_GUI.calibrations,))
thread.start()

In [ ]:
# Clean shutdown
cmdsend.send(['ShutDown',''])